<a href="https://colab.research.google.com/github/zjx1123/zjx/blob/master/PhysGen_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PhysGen Demo notebook
Demo note book for "PhysGen: Rigid-Body Physics-Grounded Image-to-Video Generation", Shaowei Liu, Zhongzheng Ren, Saurabh Gupta, Shenlong Wang, in ECCV 2024

A novel **training-free** image-to-video generation pipeline integrates physical simulation and generative video diffusion prior.

Project webpage: https://stevenlsw.github.io/physgen/

Github repo: https://github.com/stevenlsw/physgen

## Instructions

Please **make a copy of this notebook or open in playground mode** and make changes there.

This notebook provides a step by step guide to perform the following:
- A. Enable GPU (Runtime > Change Runtime Type > GPU)
- B. Get code and setup environment
- C. Run Demo

# A. Enable GPU

Go to Runtime > Change Runtime Type > GPU

# B. Get code and setup environment

### Clone the github repository

In [20]:
%cd /content/
!git clone --recurse-submodules https://github.com/stevenlsw/physgen

/content
fatal: destination path 'physgen' already exists and is not an empty directory.


## Setup the environment

In [21]:
%cd physgen/
!pip install -r requirements.txt

/content/physgen
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 32.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [2]:
!pip install tokenizers

In [5]:
!pip install kornia

In [1]:
!pip install xformers --pre --no-build-isolation

In [12]:
!pip install pymunk

  Using cached pymunk-7.1.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (7.2 kB)
Using cached pymunk-7.1.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.0 MB)


# C.  Run demo

Choose a demo name from "pool", "car", "domino", "pig_ball", "balls"

In [6]:
!export name="domino"

In [7]:
# match above demo name for python environment
name = "domino"

## Step-1: Run image space dynamics simulation


In [13]:
!mkdir ../outputs
%cd /content/physgen/simulation
!python animate.py --data_root ../data --save_root ../outputs --config ../data/$name/sim.yaml
%cd ../

mkdir: cannot create directory ‘../outputs’: File exists
/content/physgen/simulation
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
/content/physgen/simulation/animate_utils.py:176: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  center = torch.tensor([center]).float() # (1, 2)
/content/physgen


In [14]:
!pip install moviepy

## Setp-2: Run relighting

In [15]:
%cd relight
!python relight.py --perception_input ../data/$name --previous_output ../outputs/$name
%cd ../

/content/physgen/relight
/content/physgen/relight/relight_utils.py:117: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  coeffs = torch.tensor([x, y, z]).reshape(3, 1).detach().numpy()
100% 15/15 [00:02<00:00,  5.79it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
done!
/content/physgen


## Step-3: Run Video Diffusion Rendering

### Download the SEINE model

The model is used for video diffusion prior rendering.

In [16]:
# install git-lfs beforehand
!sudo apt-get install git-lfs
!git lfs install
!mkdir -p diffusion/SEINE/pretrained
!git clone https://huggingface.co/CompVis/stable-diffusion-v1-4 diffusion/SEINE/pretrained/stable-diffusion-v1-4
!wget -P diffusion/SEINE/pretrained https://huggingface.co/Vchitect/SEINE/resolve/main/seine.pt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Updated git hooks.
Git LFS initialized.
Cloning into 'diffusion/SEINE/pretrained/stable-diffusion-v1-4'...
remote: Enumerating objects: 768, done.
remote: Total 768 (delta 0), reused 0 (delta 0), pack-reused 768 (from 1)
Receiving objects: 100% (768/768), 685.42 KiB | 22.85 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Filtering content: 100% (18/18), 21.71 GiB | 38.60 MiB/s, done.
--2025-10-29 08:11:19--  https://huggingface.co/Vchitect/SEINE/resolve/main/seine.pt
Resolving huggingface.co (huggingface.co)... 18.65.25.32, 18.65.25.54, 18.65.25.19, ...
Connecting to huggingface.co (huggingface.co)|18.65.25.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/654df2c386f53e80529c9013/c986b060

In [23]:
!pip install "jax[cuda12_pip]==0.6.0" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.6.0-py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 MB 11.9 MB/s eta 0:00:00


In [33]:
!pip uninstall -y pyarrow datasets
!pip install pyarrow==19.0.0 datasets

Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0
Found existing installation: datasets 4.3.0
Uninstalling datasets-4.3.0:
  Successfully uninstalled datasets-4.3.0
  Using cached pyarrow-19.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-4.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached datasets-4.1.1-py3-none-any.whl.metadata (18 kB)
  Using cached datasets-4.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
Using cached pyarrow-19.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.1 MB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)


In [1]:
import pyarrow
import datasets
print("pyarrow版本：", pyarrow.__version__)
print("datasets版本：", datasets.__version__)

pyarrow版本： 19.0.0
datasets版本： 4.0.0


In [1]:
!pip uninstall -y datasets

Found existing installation: datasets 4.3.0
Uninstalling datasets-4.3.0:
  Successfully uninstalled datasets-4.3.0


In [26]:
!pip install torchvision

### Run generative video diffusion rendering

In [1]:
%cd /content/physgen/diffusion
!python video_diffusion.py --perception_input ../data/$name --previous_output ../outputs/$name
%cd ../

[Errno 2] No such file or directory: '/content/physgen/diffusion'
/content
python3: can't open file '/content/video_diffusion.py': [Errno 2] No such file or directory
/


In [4]:
# 搜索函数定义
!grep -r "def video_transforms" .

# 搜索模块文件（可能是一个独立的py文件）
!find . -name "video_transforms.py"

# 搜索所有包含该名称的导入或定义
!grep -r "video_transforms" . | grep -E "def |class |import "

^C
./diffusion/SEINE/datasets/video_transforms.py
^C


In [ ]:
import numpy as np
from IPython.display import HTML
from base64 import b64encode

# Function to show the video in Colab
def show_video(video_path, video_width=600):
    # Read the video file as bytes
    video_file = open(video_path, "rb").read()
    # Convert the video file to a base64 string
    video_base64 = b64encode(video_file).decode()

    # Create an HTML5 video element to display the video
    video_html = f'''
    <video width="{video_width}" controls>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
    </video>
    '''
    return HTML(video_html)

In [ ]:
video_path = f"outputs/{name}/final_video.mp4"
show_video(video_path)